In [ ]:
from scipy.ndimage.filters import gaussian_filter
import os, random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np


def segment_otsu(img):
    bins_num = 256


    # Get the image histogram
    hist, bin_edges = np.histogram(img, bins=bins_num)

    # Get normalized histogram if it is required

    hist = np.divide(hist.ravel(), hist.max())

    # Calculate centers of bins
    bin_mids = (bin_edges[:-1] + bin_edges[1:]) / 2.

    # Iterate over all thresholds (indices) and get the probabilities w1(t), w2(t)
    weight1 = np.cumsum(hist)
    weight2 = np.cumsum(hist[::-1])[::-1]

    # Get the class means mu0(t)
    mean1 = np.cumsum(hist * bin_mids) / weight1
    # Get the class means mu1(t)
    mean2 = (np.cumsum((hist * bin_mids)[::-1]) / weight2[::-1])[::-1]

    inter_class_variance = weight1[:-1] * weight2[1:] * (mean1[:-1] - mean2[1:]) ** 2

    # Maximize the inter_class_variance function val
    index_of_max_val = np.argmax(inter_class_variance)

    threshold = bin_mids[:-1][index_of_max_val]
    print("Otsu's algorithm implementation thresholding result: ", threshold)
    result = np.zeros_like(img)
    result[img > threshold] = 255
    result[img <= threshold] = 0

    return result

def lower_resolution_image (image):
    return image.resize((80,60))

def blur_image(image):
    return gaussian_filter(image, sigma=1.5)


def get_data():
    dir = "Dataset/Rock_Paper_Sissors_Photos/train/scissors"
    file = random.choice(os.listdir(dir))
    path = os.path.join(dir, file)
    print(path)
    original_image = Image.open(path)
    greyscale_image = Image.open(path).convert('L')  # convert to grayscale

    return original_image, greyscale_image

def crop_image(original, image):
    summation_row_array = np.apply_along_axis(sum,axis=0,arr=image)
    summation_column_array = np.apply_along_axis(sum,axis=1,arr=image)
    delete_indices = []
    max_row_value = 0
    for index in range(len(summation_row_array)):
        if summation_row_array[index] == max_row_value:
            delete_indices.append(index)
        else:
            break

    for index in range(len(summation_row_array)-1,-1,-1):
        if summation_row_array[index] == max_row_value:
            delete_indices.append(index)
        else:
            break
    original = np.delete(original,delete_indices,axis=1)
    image = np.delete(image,delete_indices,axis=1)

    delete_indices = []
    max_column_value = 0
    for index in range(len(summation_column_array)):
        if summation_column_array[index] == max_column_value:
            delete_indices.append(index)
        else:
            break

    for index in range(len(summation_column_array)-1,-1,-1):
        if summation_column_array[index] == max_column_value:
            delete_indices.append(index)
        else:
            break
    original = np.delete(original,delete_indices,axis=0)
    image = np.delete(image,delete_indices,axis=0)

    return original, image

def delete_background(original, segmentation):
    original[segmentation == 0] = 255
    return original

def visualize_greyscale(image, name):
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.imshow(image, cmap=plt.cm.gray)
    #ax.set_title(name)
    ax.axis('off')

def visualize_image(image,name):
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.imshow(image)
    ax.set_title(name)
    ax.axis('off')

def create_histogram(image):
    column_data = np.zeros(image.shape[0])
    for column in range(image.shape[0]):
        column_data[column] = np.sum(image[column])
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.plot(column_data, range(len(column_data)))
    ax.invert_yaxis()
    ax.invert_xaxis()
    ax.set_title('y plot')
    ax.axis('off')
    row_data = np.zeros(image.shape[1])

    for row in range(image.shape[1]):
        row_data[row] = image.sum(axis=0)[row]
    fig, ax = plt.subplots(figsize=(4, 3))
    ax.plot(range(len(row_data)), row_data)
    ax.set_title('x plot')
    ax.axis('off')


In [ ]:
def segment_simple_use_case():
    original, greyscale = get_data()
    low_resolution = lower_resolution_image(greyscale)
    blurred = blur_image(low_resolution)
    segmented = segment_otsu(blurred)
    cropped_original, cropped = crop_image(low_resolution, segmented)
    background_deletion = delete_background(cropped_original, cropped)
    create_histogram(cropped)

    visualize_image(original,"Original image")
    visualize_greyscale(greyscale, "Grayscaling")
    visualize_greyscale(low_resolution, "Resolution reduction")
    visualize_greyscale(blurred, "Blurring")
    visualize_greyscale(segmented, "Segmentation")
    visualize_greyscale(cropped, "Centring and zooming in")
    visualize_greyscale(background_deletion, "original without background")



In [ ]:
segment_simple_use_case()